In [1]:
import random
import matplotlib.pyplot as plt
import time
import numpy as np
%matplotlib tk

## Struktury danych

In [2]:
class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y
    def __str__(self):
        return "Point(%s,%s)"%(self.x,self.y)

In [3]:
class Segment:
    def __init__(self,p,q):
        
        if p.x < q.x:
            left, right=p, q
        else:
            left, right=q, p
                    
        self.left=left
        self.right=right
        self.a=(right.y-left.y)/(right.x-left.x) 
        self.b=left.y-left.x*self.a
        
    def get_y(self,x):
        return self.a*x+self.b


In [4]:
class Trapezoid:
    def __init__(self, top, bottom, leftp, rightp):

        self.top = top
        self.bottom = bottom
        self.leftp = leftp
        self.rightp = rightp
        
        self.dnode = None # wskaznik do węzła w grafie wyszukiwań

        # wskazniki do sasiadow
        self.upper_left = None
        self.lower_left = None
        self.upper_right = None
        self.lower_right = None


In [5]:
class DNode:
    def __init__(self, node_type, label, left=None, right=None, above = None, below = None):
        """
        Inicjalizuje węzeł w drzewie/grafie, który może reprezentować współrzędne ('x', 'y') lub trapez ('leaf').
        Atrybut 'parents' jest używany tylko dla trapezów w grafie wyszukiwań, aby śledzić rodziców.
        Pozostałe atrybuty umożliwiają powiązanie węzłów w strukturze 2D (lewy, prawy, powyżej, poniżej).
        """
        self.node_type = node_type  # Typ węzła: 'x', 'y', 'leaf'
        self.label = label          # Punkt, odcinek lub trapez
        self.left = left            # Lewy potomek
        self.right = right          # Prawy potomek
        self.above = above
        self.below = below
        self.parents=[]  # lista rodziców potrzebna do zastępowania trapezów w grafie wyszukiwań na inne węzły
        # potrzebne tylko w przypadku trapezów
        
       
    def replace_node(self, new_node):
        """
        Zastępuje bieżący węzeł nowym w każdym z jego rodziców, aktualizując odpowiednie referencje (lewy, prawy, powyżej, poniżej).
        Nowy węzeł zostaje dodany do listy rodziców, a stare połączenia są usuwane.
        Funkcja jest przydatna w operacjach zastępowania węzłów w strukturze grafu.
        """
        for parent in self.parents:
            if parent.left == self:
                parent.left = new_node
            elif parent.above == self:
                parent.above = new_node
            elif parent.below == self:
                parent.below = new_node
            else:
                parent.right = new_node
            new_node.parents.append(parent)

    
class DTree:
    def __init__(self):
        self.root = None

    def query(self, point,a=None):
        """
        Znajduje trapez, który zawiera dany punkt.
        point: (x, y) - współrzędne punktu
        Zwraca: Trapez (etykieta liścia), który zawiera punkt
        """
        current = self.root
        while current:
            if current.node_type == 'x':
                if point.x < current.label.x:
                    current = current.left                    
                else:
                    current = current.right
            elif current.node_type == 'y':
                segment = current.label
                y_on_line = segment.get_y(point.x)
                if point.y > y_on_line:
                    current = current.above 
                elif point.y==y_on_line and a is not None:
                    if segment.a>a:
                        current = current.below
                    else:
                        current = current.above
                else:
                    current = current.below
            elif current.node_type == 'leaf':
                return current.label

        return None

## Generator losowych odcinków
Wylosowane odcinki są w położeniu ogólnym i spełniają założenia algorytmu

In [6]:
import random

def segments_intersect(s1, s2):

    def ccw(a, b, c):
        """Sprawdza orientację (a, b, c): czy jest zgodna z ruchem wskazówek zegara."""
        return (c[1] - a[1]) * (b[0] - a[0]) - (b[1] - a[1]) * (c[0] - a[0])>1e-10
    
    (a1, a2), (b1, b2) = s1, s2
    return (ccw(a1, b1, b2) != ccw(a2, b1, b2)) and (ccw(a1, a2, b1) != ccw(a1, a2, b2))

def segment_contains_point(s, p):
    """
    Sprawdza, czy odcinek s zawiera punkt p (punkt p może leżeć na końcu odcinka, dlatego wtedy funkcja zwraca false).
    """
    (x1, y1), (x2, y2) = s
    if p>(min(x1,x2),min(y1,y2)) and p<(max(x1,x2),max(y1,y2)) and abs((x1-x2)*(p[1]-y1)-(y1-y2)*(p[0]-x1))<1e-10: # sprawdzam czy punkt leży na prostej zawierającej odcinek
        return True
    return False

def generate_segments(n, x_range, y_range):

    segments = []

    for _ in range(n):
        while True:
            x1 = random.uniform(x_range[0], x_range[1])
            y1 = random.uniform(y_range[0], y_range[1])
            x2 = random.uniform(x_range[0], x_range[1])
            y2 = random.uniform(y_range[0], y_range[1])

            if x1== x2:
                continue
            
            new_segment = ((x1, y1), (x2, y2))
            
            if all(not segments_intersect(new_segment, segment) for segment in segments) and all(not segment_contains_point(segment, (x1, y1)) and not segment_contains_point(segment, (x2, y2)) for segment in segments):
                segments.append(new_segment)
                break

    return segments

Aby wygenerować losowe odcinki uruchom poniższą komórkę dla odpowiednich argumentów.

In [7]:
lines=generate_segments(10,(0,10),(0,10))

## Aplikacja do zadawania odcinków

Aby wprowadzić odcinek należy nacisnąć lewym przyciskiem myszy w punkcie początkowym, przeciągnąć kursor i zwolnić przycisk w punkcie końcowym.
Po wprowadzeniu wszystkich odcinków wystarczy zamknąć okno. Jeśli odcinki są wprowadzane losowo, należy pominąć poniższą komórkę.

In [8]:
class LineDrawer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.lines = []
        self.current_line = None
        self.start_point = None

        # stałe wymiary okna (można zmienić)
        self.ax.set_xlim(0, 10)
        self.ax.set_ylim(0, 10)

        self.cid_click = self.fig.canvas.mpl_connect('button_press_event', self.on_click)
        self.cid_motion = self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)
        self.cid_release = self.fig.canvas.mpl_connect('button_release_event', self.on_release)

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        self.start_point = (event.xdata, event.ydata)
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = self.ax.plot([event.xdata, event.xdata], [event.ydata, event.ydata], 'k-')[0]

    def on_motion(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        xdata, ydata = self.current_line.get_data()
        xdata[1], ydata[1] = event.xdata, event.ydata
        self.current_line.set_data(xdata, ydata)
        self.fig.canvas.draw()

    def on_release(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        self.lines.append((self.start_point, (event.xdata, event.ydata)))
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = None
        self.start_point = None
        self.fig.canvas.draw()

    def show(self):
        plt.show()

    def get_lines(self):
        return self.lines

drawer = LineDrawer()
drawer.show()
lines=drawer.get_lines()


Poniższa funkcja na podstawie zadanej listy odcinków w postaci krotek zwraca listę obiektów klasy Segment.

In [9]:
# zamiana reprezentacji odcinków
def change_segment_representation(lines):
    s=[0 for _ in range(len(lines))]
    for i in range(len(lines)):
        p=Point(lines[i][0][0],lines[i][0][1])
        q=Point(lines[i][1][0],lines[i][1][1])
        segment=Segment(p,q)
        s[i]=segment
    return s

Po wygenerowaniu odcinków losowo lub ręcznie, należy uruchomić poniższą komórkę.

In [10]:
segments = change_segment_representation(lines)

## Konstrukcja mapy trapezowej T(S) - FUNKCJE POMOCNICZE

In [11]:
def find_area(D,segment):
    
    """
    Znajduje obszar w którym znajduje się dany segment, śledząc trapezy w drzewie.
    Zwraca listę trapezów, które zawierają segment, aż do osiągnięcia prawego końca.
    """
    
    area=[]
    p=segment.left
    q=segment.right
    delta=D.query(p,segment.a)
    area.append(delta)
    while q.x>delta.rightp.x:
        if delta.rightp.y>segment.get_y(delta.rightp.x):
            area.append(delta.lower_right)
            delta=delta.lower_right
        else:
            area.append(delta.upper_right)
            delta=delta.upper_right
    return area

In [12]:
def update_left(old_trap, left, top, bottom):
    """
    Aktualizuje połączenia w lewo w strukturze trapezu.
    Jeśli istnieje trapez po lewej stronie, łączy go z nowym trapezem, w przeciwnym przypadku ustawia połączenia dla górnego i dolnego trapezu.
    """
    if left:
        if old_trap.upper_left:
            if old_trap.upper_left.upper_right == old_trap:
                old_trap.upper_left.upper_right = left
            if old_trap.upper_left.lower_right == old_trap:
                old_trap.upper_left.lower_right = left
        if old_trap.lower_left:
            if old_trap.lower_left.upper_right == old_trap:
                old_trap.lower_left.upper_right = left
            if old_trap.lower_left.lower_right == old_trap:
                old_trap.lower_left.lower_right = left
    else:
        if old_trap.upper_left:
            old_trap.upper_left.upper_right = top
        if old_trap.lower_left:
            old_trap.lower_left.lower_right = bottom


In [13]:
def update_right(old_trap, right, top, bottom):
    """
    Aktualizuje połączenia w prawo w strukturze trapezu.
    Jeśli istnieje trapez po prawej stronie, łączy go z nowym trapezem, w przeciwnym przypadku ustawia połączenia dla górnego i dolnego trapezu.
    """
    if right:
        if old_trap.upper_right:
            if old_trap.upper_right.upper_left == old_trap:
                old_trap.upper_right.upper_left = right
            if old_trap.upper_right.lower_left == old_trap:
                old_trap.upper_right.lower_left = right
        if old_trap.lower_right:
            if old_trap.lower_right.upper_left == old_trap:
                old_trap.lower_right.upper_left = right
            if old_trap.lower_right.lower_left == old_trap:
                old_trap.lower_right.lower_left = right
    else:
        if old_trap.upper_right:
            old_trap.upper_right.upper_left = top
        if old_trap.lower_right:
            old_trap.lower_right.lower_left = bottom

In [14]:
def shuffle_segments(segments):
    """
    Tasuje listę segmentów w sposób losowy.
    Zwraca posortowaną losowo listę segmentów.
    """
    random.shuffle(segments)
    return segments

In [15]:
def outer_trapezoid(segments):
    """
    Tworzy trapez, który otacza wszystkie odcinki, bazując na ekstremalnych wartościach współrzędnych (x, y).
    Zwraca trapez, którego górna i dolna krawędź są ustawione na podstawie minimalnych i maksymalnych punktów z segmentów.
    """
    # Inicjalizujemy zmienne, które będą przechowywać minimalne i maksymalne wartości
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    # Przechodzimy przez wszystkie odcinki, aby znaleźć ekstremalne punkty
    for (x1, y1), (x2, y2) in segments:
        min_x = min(min_x, x1, x2)
        max_x = max(max_x, x1, x2)
        min_y = min(min_y, y1, y2)
        max_y = max(max_y, y1, y2)

    upper_segment=Segment(Point(min_x-1, max_y+1),Point(max_x+1, max_y+1))
    lower_segment=Segment(Point(min_x-1, min_y-1),Point(max_x+1, min_y-1))
    trapezoid=Trapezoid(upper_segment,lower_segment,Point(min_x-1, min_y-1),Point(max_x+1, max_y+1))
    
    return trapezoid

## Konstrukcja mapy trapezowej T(S) - FUNKCJE DODAWANIA ODCINKA

In [16]:
def insert_into_one_trapezoid(dtree, segment, trapezoid):
    """
    Wstawia nowy segment do jednego trapezu, dzieląc go na cztery nowe trapezy.
    Tworzy nowe węzły w drzewie (DTree), aktualizuje połączenia między trapezami oraz aktualizuje strukturę drzewa.
    Zwraca zbiór nowo utworzonych trapezów.
    """
    added_traps=set()

    if segment.left.x > trapezoid.leftp.x:
        left_trapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, trapezoid.leftp, segment.left)
        added_traps.add(left_trapezoid)
    else:
        left_trapezoid = None
    
    if segment.right.x < trapezoid.rightp.x:
        right_trapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, segment.right, trapezoid.rightp)
        added_traps.add(right_trapezoid)
    else:
        right_trapezoid = None
        
    top_trapezoid = Trapezoid(trapezoid.top, segment, segment.left, segment.right)
    bottom_trapezoid = Trapezoid(segment, trapezoid.bottom, segment.left, segment.right)
    added_traps.add(top_trapezoid)
    added_traps.add(bottom_trapezoid)

    
    if left_trapezoid:
        left_trapezoid.upper_right = top_trapezoid
        left_trapezoid.lower_right = bottom_trapezoid
        left_trapezoid.upper_left=trapezoid.upper_left
        left_trapezoid.lower_left=trapezoid.lower_left
        top_trapezoid.upper_left = left_trapezoid
        bottom_trapezoid.lower_left = left_trapezoid
        
    else:
        top_trapezoid.upper_left = trapezoid.upper_left
        bottom_trapezoid.lower_left = trapezoid.lower_left
        
    update_left(trapezoid, left_trapezoid, top_trapezoid, bottom_trapezoid)
    
    if right_trapezoid:
        right_trapezoid.upper_left = top_trapezoid
        right_trapezoid.lower_left = bottom_trapezoid
        right_trapezoid.upper_right=trapezoid.upper_right
        right_trapezoid.lower_right=trapezoid.lower_right
        top_trapezoid.upper_right = right_trapezoid
        bottom_trapezoid.lower_right = right_trapezoid

    else:
        top_trapezoid.upper_right = trapezoid.upper_right
        bottom_trapezoid.lower_right = trapezoid.lower_right

    update_right(trapezoid, right_trapezoid, top_trapezoid, bottom_trapezoid)   

    # zmiany w dtree
    leftpoint=DNode('x',segment.left)
    if left_trapezoid:
        left=left_trapezoid
    else:
        if trapezoid.upper_left:
            left=trapezoid.upper_left
        else:
            left=trapezoid.lower_left
    leftpoint.left=DNode('leaf',left)
    if left_trapezoid:
        left.dnode=leftpoint.left
    if left:
        left.dnode.parents.append(leftpoint)
    rightpoint=DNode('x',segment.right)
    leftpoint.right=rightpoint
    if right_trapezoid:
        right=right_trapezoid
    else:
        if trapezoid.upper_right:
            right=trapezoid.upper_right
        else:
            right=trapezoid.lower_right
    rightpoint.right=DNode('leaf',right)
    if right_trapezoid:
        right.dnode=rightpoint.right
    if right:
        right.dnode.parents.append(rightpoint)
    segment_node=DNode('y',segment)
    rightpoint.left=segment_node
    
    segment_node.above=DNode('leaf',top_trapezoid)
    top_trapezoid.dnode=segment_node.above
    top_trapezoid.dnode.parents.append(segment_node)
    
    segment_node.below=DNode('leaf',bottom_trapezoid)
    bottom_trapezoid.dnode=segment_node.below
    bottom_trapezoid.dnode.parents.append(segment_node)
    if dtree.root==trapezoid.dnode:
        dtree.root=leftpoint
    else:
        trapezoid.dnode.replace_node(leftpoint)
    return added_traps

Wstawianie odcinka, jeżeli w area jest więcej niż 1 trapez

In [17]:
def process_first_trapezoid(trapezoid, segment, new_trapezoids):
    left_trapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, trapezoid.leftp, segment.left)
    if trapezoid.rightp.y > segment.get_y(trapezoid.rightp.x):
        upper_trap = Trapezoid(trapezoid.top, segment, segment.left, trapezoid.rightp)
        lower_trap = Trapezoid(segment, trapezoid.bottom, segment.left, trapezoid.rightp)
        merge_upper = False
    else:
        upper_trap = Trapezoid(trapezoid.top, segment, segment.left, trapezoid.rightp)
        lower_trap = Trapezoid(segment, trapezoid.bottom, segment.left, trapezoid.rightp)
        merge_upper = True

    left_trapezoid.upper_right = upper_trap
    left_trapezoid.upper_left = trapezoid.upper_left
    left_trapezoid.lower_right = lower_trap
    left_trapezoid.lower_left = trapezoid.lower_left

    if merge_upper:
        upper_trap.upper_right = trapezoid.upper_right
        upper_trap.lower_right = trapezoid.upper_right
        lower_trap.upper_right = trapezoid.upper_right 
        lower_trap.lower_right = trapezoid.lower_right
    else:
        upper_trap.upper_right = trapezoid.upper_right
        upper_trap.lower_right = trapezoid.lower_right
        lower_trap.upper_right = trapezoid.lower_right
        lower_trap.lower_right = trapezoid.lower_right

    upper_trap.upper_left = left_trapezoid
    upper_trap.lower_left = left_trapezoid
    lower_trap.upper_left= left_trapezoid
    lower_trap.lower_left = left_trapezoid

    update_left(trapezoid, left_trapezoid, upper_trap, lower_trap)
    if merge_upper:
        update_right(trapezoid, lower_trap, upper_trap, lower_trap)
    else:
        update_right(trapezoid, upper_trap, lower_trap, upper_trap)

    new_trapezoids.append(left_trapezoid)
    new_trapezoids.append(upper_trap)
    new_trapezoids.append(lower_trap)

    left_node = DNode('leaf', left_trapezoid)
    left_trapezoid.dnode = left_node
    upper_node = DNode('leaf', upper_trap)
    upper_trap.dnode = upper_node
    lower_node = DNode('leaf', lower_trap)
    lower_trap.dnode = lower_node

    s_node = DNode('y', segment, above=upper_node, below=lower_node)

    p_node = DNode('x', segment.left, left=left_node, right=s_node)

    left_node.parents.append(p_node)
    upper_node.parents.append(s_node)
    lower_node.parents.append(s_node)

    trapezoid.dnode.replace_node(p_node)

    new_trapezoids.extend([left_trapezoid, upper_trap, lower_trap])

    return upper_trap, lower_trap, merge_upper


def process_middle_trapezoid(trapezoid, segment, upper_trap, lower_trap, merge_upper, new_trapezoids):
    changed = False

    if merge_upper:
        prev_lower_trap = lower_trap
        lower_trap = Trapezoid(segment, trapezoid.bottom, trapezoid.leftp, trapezoid.rightp)
        upper_node = upper_trap.dnode
        upper_trap.dnode = upper_node
        lower_node = DNode('leaf', lower_trap)
        lower_trap.dnode = lower_node
    else:
        prev_upper_trap = upper_trap
        upper_trap = Trapezoid(trapezoid.top, segment, trapezoid.leftp, trapezoid.rightp)
        upper_node = DNode('leaf', upper_trap)
        upper_trap.dnode = upper_node
        lower_node = lower_trap.dnode
        lower_trap.dnode = lower_node

    if trapezoid.rightp.y > segment.get_y(trapezoid.rightp.x):
        upper_trap.rightp = trapezoid.rightp
        if merge_upper:
            changed = True
        merge_upper = False
        new_trapezoids.append(upper_trap)
    else:
        lower_trap.rightp = trapezoid.rightp
        if not merge_upper:
            changed = True 
        merge_upper = True
        new_trapezoids.append(lower_trap)

    if merge_upper:
        if changed:
            upper_trap.upper_right = trapezoid.upper_right
            upper_trap.lower_right = trapezoid.upper_right
            upper_trap.upper_left = trapezoid.upper_left
            upper_trap.lower_left = prev_upper_trap

            lower_trap.upper_right = trapezoid.upper_right
            lower_trap.lower_right = trapezoid.lower_right

            update_left(trapezoid, upper_trap, upper_trap, lower_trap) 
            update_right(trapezoid, lower_trap, upper_trap, lower_trap) 
        else:
            upper_trap.upper_right = trapezoid.upper_right
            upper_trap.lower_right = trapezoid.upper_right

            lower_trap.upper_right = trapezoid.upper_right
            lower_trap.lower_right = trapezoid.lower_right
            lower_trap.upper_left = prev_lower_trap
            lower_trap.lower_left = trapezoid.lower_left

            update_left(trapezoid, lower_trap, upper_trap, lower_trap)  
            update_right(trapezoid, lower_trap, upper_trap, lower_trap)
    else:
        if changed:
            upper_trap.upper_right = trapezoid.upper_right
            upper_trap.lower_right = trapezoid.lower_right

            lower_trap.upper_right = trapezoid.lower_right
            lower_trap.lower_right = trapezoid.lower_right
            lower_trap.upper_left = prev_lower_trap
            lower_trap.lower_left = trapezoid.lower_left

            update_left(trapezoid, lower_trap, upper_trap, lower_trap)
            update_right(trapezoid, upper_trap, upper_trap, lower_trap)
        else:
            upper_trap.upper_right = trapezoid.upper_right
            upper_trap.upper_left = trapezoid.upper_left
            upper_trap.lower_right = trapezoid.lower_right
            upper_trap.lower_left = prev_upper_trap

            lower_trap.upper_right = trapezoid.lower_right
            lower_trap.lower_right = trapezoid.lower_right

            update_left(trapezoid, upper_trap, upper_trap, lower_trap) 
            update_right(trapezoid, upper_trap, upper_trap, lower_trap)

    s_node = DNode('y', segment, above=upper_node, below=lower_node)
    upper_node.parents.append(s_node)
    lower_node.parents.append(s_node)
    trapezoid.dnode.replace_node(s_node)

    return upper_trap, lower_trap, merge_upper


def process_last_trapezoid(trapezoid, segment, upper_trap, lower_trap, merge_upper, new_trapezoids):
    right_trap = Trapezoid(trapezoid.top, trapezoid.bottom, segment.right, trapezoid.rightp)
    if merge_upper:
        upper_trap.rightp = segment.right
        lower_trap = Trapezoid(segment, trapezoid.bottom, trapezoid.leftp, segment.right)
        upper_node = upper_trap.dnode
        upper_trap.dnode = upper_node
        lower_node = DNode('leaf', lower_trap)
        lower_trap.dnode = lower_node
    else:
        upper_trap = Trapezoid(trapezoid.top, segment, trapezoid.leftp, segment.right)
        lower_trap.rightp = segment.right
        upper_node = DNode('leaf', upper_trap)
        upper_trap.dnode = upper_node
        lower_node = lower_trap.dnode
        lower_trap.dnode = lower_node

    right_trap.upper_right = trapezoid.upper_right
    right_trap.upper_left = upper_trap
    right_trap.lower_right = trapezoid.lower_right
    right_trap.lower_left = lower_trap

    upper_trap.upper_right = right_trap
    upper_trap.lower_right = right_trap
    lower_trap.upper_right = right_trap
    lower_trap.lower_right = right_trap

    if merge_upper:
        lower_trap.upper_left = trapezoid.upper_left
        lower_trap.lower_left = trapezoid.lower_left
    else:
        upper_trap.upper_left = trapezoid.upper_left
        upper_trap.lower_left = trapezoid.lower_left

    update_right(trapezoid, right_trap, upper_trap, lower_trap)
    if merge_upper:
        update_left(trapezoid, lower_trap, upper_trap, lower_trap) 
    else:
        update_left(trapezoid, upper_trap, upper_trap, lower_trap) 

    new_trapezoids.append(upper_trap)
    new_trapezoids.append(lower_trap)
    new_trapezoids.append(right_trap)

    right_node = DNode('leaf', right_trap)
    right_trap.dnode = right_node

    s_node = DNode('y', segment, above=upper_node, below=lower_node)
    upper_node.parents.append(s_node)
    lower_node.parents.append(s_node)

    q_node = DNode('x', segment.right, left=s_node, right=right_node)
    right_node.parents.append(q_node)

    trapezoid.dnode.replace_node(q_node)

    new_trapezoids.extend([upper_trap, lower_trap, right_trap])


def insert_into_many_trapezoids(search_graph, intersecting_trapezoids, segment):
    upper_trap, lower_trap, merge_upper = None, None, False
    new_trapezoids = []

    for index, trapezoid in enumerate(intersecting_trapezoids):
        if index == 0:
            upper_trap, lower_trap, merge_upper = process_first_trapezoid(trapezoid, segment, new_trapezoids)
        elif index == len(intersecting_trapezoids) - 1:
            process_last_trapezoid(trapezoid, segment, upper_trap, lower_trap, merge_upper, new_trapezoids)
        else:
            upper_trap, lower_trap, merge_upper = process_middle_trapezoid(
                trapezoid, segment, upper_trap, lower_trap, merge_upper, new_trapezoids)

    return set(new_trapezoids)




## Konstrukcja mapy trapezowej T(S) - FUNKCJA GŁÓWNA

In [18]:
def build_trapezoidal_map(lines):
    
    """
    Funkcja 'uruchamiająca'.
    Tworzy drzewo trapezów, dodaje segmenty do mapy trapezów oraz aktualizuje strukturę drzewa.
    Zaczyna od inicjalizacji korzenia drzewa i wstawiania segmentów do odpowiednich trapezów.
    """
    
    o_trapezoid = outer_trapezoid(lines)
    
    # Tworzenie drzewa
    dtree = DTree()
    
    # Tworzenie korzenia
    x_node = DNode('leaf', o_trapezoid)
    o_trapezoid.dnode = x_node
    dtree.root = x_node
    
    segments = shuffle_segments(change_segment_representation(lines))
    for segment in segments:

        area = find_area(dtree, segment)

        n = len(area)
        if n == 1:
            new=insert_into_one_trapezoid(dtree, segment, area[0])
        elif n > 1:
            new=insert_into_many_trapezoids(dtree, area, segment)
            
    return dtree

## Wizualizatory:

#### WIZUALIZATOR ODCINKÓW I TRAPEZU ZEWNĘTRZNEGO:

In [19]:
def visualize_segments_and_trapezoid(segments):
    """
    Rysuje segmenty oraz trapez otaczający je na wykresie.
    Segmenty są wyświetlane na niebiesko, a trapez na czerwono, z przerywaną linią.
    """
    trapezoid = outer_trapezoid(segments)
    
    plt.figure(figsize=(8, 6))
    for (x1, y1), (x2, y2) in segments:
        plt.plot([x1, x2], [y1, y2], marker='o', color='b', label='Odcinek' if (x1, y1) == segments[0][0] else "")

    trapezoid_x = [trapezoid.leftp.x, trapezoid.rightp.x, trapezoid.rightp.x, trapezoid.leftp.x, trapezoid.leftp.x]
    trapezoid_y = [trapezoid.top.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.rightp.x), trapezoid.bottom.get_y(trapezoid.rightp.x), trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.leftp.x)]
    plt.plot(trapezoid_x, trapezoid_y, color='r', label='Trapez', linestyle='--')
    
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Wizualizacja odcinków i trapezu')
    plt.grid(True)
    plt.legend()
    plt.show()

In [20]:
visualize_segments_and_trapezoid(lines)

#### WIZUALIZATOR MAPY TRAPEZOWEJ (ANIMACJA):

In [21]:
def visualise_map_construction(lines):
    
    """
    Wizualizuje proces konstrukcji mapy trapezów, rysując kolejne etapy dodawania segmentów do mapy.
    Na wykresie segmenty są rysowane na czerwono, a trapezy na zielono. Proces jest pokazany z animacją.
    """

    o_trapezoid = outer_trapezoid(lines)
    segments = change_segment_representation(lines)
    shuffle_segments(segments)

    # Funkcja pomocnicza do rysowania trapezoidów
    def plot_trapezoid(trapezoid, color='g', linestyle='-'):
        plt.plot([trapezoid.leftp.x, trapezoid.leftp.x], [trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.leftp.x)], color=color,linestyle=linestyle)
        plt.plot([trapezoid.rightp.x, trapezoid.rightp.x], [trapezoid.bottom.get_y(trapezoid.rightp.x), trapezoid.top.get_y(trapezoid.rightp.x)], color=color,linestyle=linestyle)

    def draw_outer_trapezoid():
        o_trapezoid_x = [o_trapezoid.leftp.x, o_trapezoid.rightp.x, o_trapezoid.rightp.x, o_trapezoid.leftp.x, o_trapezoid.leftp.x]
        o_trapezoid_y = [o_trapezoid.top.get_y(o_trapezoid.leftp.x), o_trapezoid.top.get_y(o_trapezoid.rightp.x), o_trapezoid.bottom.get_y(o_trapezoid.rightp.x), o_trapezoid.bottom.get_y(o_trapezoid.leftp.x), o_trapezoid.top.get_y(o_trapezoid.leftp.x)]
        plt.plot(o_trapezoid_x, o_trapezoid_y, color='g', linestyle='-')

    # Inicjalizacja wykresu
    plt.figure()
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Wizualizacja etapów konstrukcji mapy')
    draw_outer_trapezoid()

    # Tworzenie drzewa
    dtree = DTree()
    # Tworzenie korzenia
    x_node = DNode('leaf', o_trapezoid)
    o_trapezoid.dnode = x_node
    dtree.root = x_node
    # Zbiór trapezów należących do mapy
    T = set([o_trapezoid])
    S=set()

    for segment in segments:

        area = find_area(dtree, segment)

        # Aktualizacja wykresu po wstawieniu segmentu
        plt.draw()
        plt.pause(1)  # Pauza, aby zobaczyć zmiany

        # Patrzymy ile trapezów zajmuje
        n = len(area)
        if n == 1:
            new=insert_into_one_trapezoid(dtree, segment, area[0])
        elif n > 1:
            new=insert_into_many_trapezoids(dtree, area, segment)

        # Rysowanie segmentu
        plt.plot([segment.left.x, segment.right.x], [segment.left.y, segment.right.y],marker='o', color='r')
        plt.pause(1)
        # Usunięcie poprzedniego wykresu i rysowanie nowych trapezoidów po wstawieniu segmentu
        plt.clf()
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('Wizualizacja etapów konstrukcji mapy')
        # Rysowanie segmentu
        plt.plot([segment.left.x, segment.right.x], [segment.left.y, segment.right.y],marker='o', color='r')
        draw_outer_trapezoid()

        T-=set(area)
        T|=new
        for trapezoid in T:
            plot_trapezoid(trapezoid, color='g')
        for s in S:
            plt.plot([s.left.x, s.right.x], [s.left.y, s.right.y],marker='o', color='b')
        plt.draw()
        plt.pause(1)  # Pauza, aby zobaczyć zmiany
        plt.plot([segment.left.x, segment.right.x], [segment.left.y, segment.right.y], marker='o',color='b')
        S.add(segment)

    # Końcowe wyświetlenie
    draw_outer_trapezoid()
    plt.show()

In [22]:
visualise_map_construction(lines)

#### WIZUALIZATOR GRAFU WYSZUKIWAŃ:
(konieczne jest zainstalowanie pakietu graphviz)

In [23]:
from graphviz import Digraph
def visualize_tree(node, graph=None):
    
    """
    Rysuje drzewo w postaci wykresu przy użyciu Graphviz.
    Węzły są reprezentowane jako trapezy, punkty lub segmenty, a krawędzie łączą je zgodnie z strukturą drzewa.
    """
    
    def format_segment(segment):
        """Formatuje segment jako dwa punkty: left i right."""
        return f"left: {segment.left}\nright: {segment.right}"
    
    if graph is None:
        graph = Digraph()
        graph.node_attr.update(shape="ellipse")
    
    if node is None:
        return graph

    # Sprawdzamy typ węzła
    if node.node_type == 'leaf':  # Węzeł liścia (trapez)
        # Pobieramy dane o trapezie
        trapezoid = node.label
        label = f"top: {format_segment(trapezoid.top)}\n" \
                f"bottom: {format_segment(trapezoid.bottom)}\n" \
                f"leftp: {trapezoid.leftp}\n" \
                f"rightp: {trapezoid.rightp}"
        graph.node(str(id(node)), label=label, shape="box")
    elif node.node_type == 'x':  # Podział względem osi X (punkt)
        label = f"x: {node.label}"
        graph.node(str(id(node)), label=label)
    elif node.node_type == 'y':  # Podział względem osi Y (segment)
        # Wyświetlamy segment jako dwa punkty (left, right)
        segment = node.label
        label = f"Segment:\n{format_segment(segment)}"
        graph.node(str(id(node)), label=label)

    # Rekurencyjnie dodajemy krawędzie i potomków
    if node.left:
        graph.edge(str(id(node)), str(id(node.left)))
        visualize_tree(node.left, graph)
    if node.right:
        graph.edge(str(id(node)), str(id(node.right)))
        visualize_tree(node.right, graph)
    if node.above:
        graph.edge(str(id(node)), str(id(node.above)))
        visualize_tree(node.above, graph)
    if node.below:
        graph.edge(str(id(node)), str(id(node.below)))
        visualize_tree(node.below, graph)

    return graph

def save_dtree_visualization(dtree, filename="dtree_visualization"):
    graph = visualize_tree(dtree.root)
    graph.render(filename, view=True)

In [24]:
dtree = build_trapezoidal_map(lines)
save_dtree_visualization(dtree)

## Lokalizator zadanego punktu:


In [25]:
def find_point(dtree, point):
    """
    Znajduje trapez zawierający dany punkt w mapie trapezów.
    """
    trapezoid = dtree.query(point)
    return trapezoid

#### Wizualizacja:

In [26]:
def visualize_map_point(point):
    
    """
    Wizualizuje mapę trapezów wraz z zadanym punktem. Zaznaczony jest również trapez, znaleziony za pomocą funkcji find_point.
    """

    o_trapezoid = outer_trapezoid(lines)
    segments = change_segment_representation(lines)
    shuffle_segments(segments)

    # Funkcja pomocnicza do rysowania trapezów
    def plot_trapezoid(trapezoid, color='g', linestyle='-'):
        plt.plot([trapezoid.leftp.x, trapezoid.leftp.x], 
                 [trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.leftp.x)], 
                 color=color, linestyle=linestyle)
        plt.plot([trapezoid.rightp.x, trapezoid.rightp.x], 
                 [trapezoid.bottom.get_y(trapezoid.rightp.x), trapezoid.top.get_y(trapezoid.rightp.x)], 
                 color=color, linestyle=linestyle)
        plt.plot([trapezoid.leftp.x, trapezoid.rightp.x], 
                 [trapezoid.top.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.rightp.x)], 
                 color=color, linestyle=linestyle)
        plt.plot([trapezoid.leftp.x, trapezoid.rightp.x], 
                 [trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.bottom.get_y(trapezoid.rightp.x)], 
                 color=color, linestyle=linestyle)
        

    def draw_outer_trapezoid():
        o_trapezoid_x = [o_trapezoid.leftp.x, o_trapezoid.rightp.x, o_trapezoid.rightp.x, o_trapezoid.leftp.x, o_trapezoid.leftp.x]
        o_trapezoid_y = [o_trapezoid.top.get_y(o_trapezoid.leftp.x), o_trapezoid.top.get_y(o_trapezoid.rightp.x), 
                         o_trapezoid.bottom.get_y(o_trapezoid.rightp.x), o_trapezoid.bottom.get_y(o_trapezoid.leftp.x), 
                         o_trapezoid.top.get_y(o_trapezoid.leftp.x)]
        plt.plot(o_trapezoid_x, o_trapezoid_y, color='magenta', linestyle='-')

    # Inicjalizacja wykresu
    plt.figure()
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Wizualizacja mapy trapezów z punktem')
    draw_outer_trapezoid()

    # Tworzenie drzewa
    dtree = DTree()
    # Tworzenie korzenia
    x_node = DNode('leaf', o_trapezoid)
    o_trapezoid.dnode = x_node
    dtree.root = x_node
    # Zbiór trapezów należących do mapy
    T = set([o_trapezoid])
    S = set()

    for segment in segments:
        area = find_area(dtree, segment)

        # Patrzymy ile trapezów zajmuje
        n = len(area)
        if n == 1:
            new = insert_into_one_trapezoid(dtree, segment, area[0])
        elif n > 1:
            new = insert_into_many_trapezoids(dtree, area, segment)


        # Aktualizacja zbiorów trapezów
        T -= set(area)
        T |= new

    # Rysowanie wszystkich trapezów
    for trapezoid in T:
        plot_trapezoid(trapezoid, color='magenta')

    # Rysowanie wszystkich segmentów
    for s in S:
        plt.plot([s.left.x, s.right.x], [s.left.y, s.right.y], marker='o', color='magenta')

    # trapez zawierający punkt
    trapezoid_with_point = find_point(dtree, point)

    if trapezoid_with_point:
        # Rysowanie punktu
        plt.plot(point.x, point.y, marker='o', color='blue', markersize=8)

        # Zaznaczenie trapezu, w którym się znajduje
        plot_trapezoid(trapezoid_with_point, color='blue', linestyle='-')


    # plt.legend()
    plt.show()




In [27]:
point = Point(4,4)
visualize_map_point(point)

In [28]:

# Funkcja do wizualizacji mapy trapezów z animacją dla punktu
# na żółto zaznaczony jest element związany z aktualnie rozpatrywanym węzłem.
# na niebiesko zaznaczony jest punkt, dla którego szukamy trapezu oraz (po zakończeniu wyszukiwania) trapez, w którym się znajduje.

def animation_map_point(lines, segments, point):
    """
    Wizualizuje mapę trapezów wraz z animacją procesu query dla zadanego punktu.
    """

    o_trapezoid = outer_trapezoid(lines)
    shuffle_segments(segments)

    fig, ax = plt.subplots()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Wizualizacja mapy trapezów z punktem')

    # Funkcja pomocnicza do rysowania trapezów
    def plot_trapezoid(trapezoid, color='g', linestyle='-'):
        ax.plot([trapezoid.leftp.x, trapezoid.leftp.x], 
                 [trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.leftp.x)], 
                 color=color, linestyle=linestyle)
        ax.plot([trapezoid.rightp.x, trapezoid.rightp.x], 
                 [trapezoid.bottom.get_y(trapezoid.rightp.x), trapezoid.top.get_y(trapezoid.rightp.x)], 
                 color=color, linestyle=linestyle)
        ax.plot([trapezoid.leftp.x, trapezoid.rightp.x], 
                 [trapezoid.top.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.rightp.x)], 
                 color=color, linestyle=linestyle)
        ax.plot([trapezoid.leftp.x, trapezoid.rightp.x], 
                 [trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.bottom.get_y(trapezoid.rightp.x)], 
                 color=color, linestyle=linestyle)

    def draw_outer_trapezoid():
        o_trapezoid_x = [o_trapezoid.leftp.x, o_trapezoid.rightp.x, o_trapezoid.rightp.x, o_trapezoid.leftp.x, o_trapezoid.leftp.x]
        o_trapezoid_y = [o_trapezoid.top.get_y(o_trapezoid.leftp.x), o_trapezoid.top.get_y(o_trapezoid.rightp.x), 
                         o_trapezoid.bottom.get_y(o_trapezoid.rightp.x), o_trapezoid.bottom.get_y(o_trapezoid.leftp.x), 
                         o_trapezoid.top.get_y(o_trapezoid.leftp.x)]
        ax.plot(o_trapezoid_x, o_trapezoid_y, color='magenta', linestyle='-')

    # Inicjalizacja wykresu
    draw_outer_trapezoid()

    # Tworzenie drzewa
    dtree = DTree()
    x_node = DNode('leaf', o_trapezoid)
    o_trapezoid.dnode = x_node
    dtree.root = x_node

    # Zbiór trapezów należących do mapy
    T = set([o_trapezoid])
    S = set()

    for segment in segments:
        area = find_area(dtree, segment)

        # Patrzymy ile trapezów zajmuje
        n = len(area)
        if n == 1:
            new = insert_into_one_trapezoid(dtree, segment, area[0])
        elif n > 1:
            new = insert_into_many_trapezoids(dtree, area, segment)

        # Aktualizacja zbiorów trapezów
        T -= set(area)
        T |= new

    # Rysowanie końcowej mapy trapezów
    ax.clear()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Wizualizacja mapy trapezów z punktem')
    draw_outer_trapezoid()
    for trapezoid in T:
        plot_trapezoid(trapezoid, color='magenta')

    # Rysowanie wszystkich segmentów
    for s in S:
        ax.plot([s.left.x, s.right.x], [s.left.y, s.right.y], marker='o', color='magenta')
    
    ax.plot(point.x, point.y, marker='o', color='blue', markersize=8)

    # Proces query dla punktu
    current_node = dtree.root
    last_line = None
    while current_node:
        if current_node.node_type == 'x':
            if last_line:
                last_line.remove()
            # Zaznaczenie punktu na osi x
            last_line, = ax.plot([current_node.label.x], [current_node.label.y], marker='o', color='orange', markersize=8)
            plt.pause(1)
            if point.x < current_node.label.x:
                current_node = current_node.left
            else:
                current_node = current_node.right
        elif current_node.node_type == 'y':
            segment = current_node.label
            y_on_line = segment.get_y(point.x)
            if last_line:
                last_line.remove()
            last_line, = ax.plot([segment.left.x, segment.right.x], [segment.left.y, segment.right.y], color='orange', linestyle='--')
            plt.pause(1)
            if point.y > y_on_line:
                current_node = current_node.above
            else:
                current_node = current_node.below
        elif current_node.node_type == 'leaf':
            if last_line:
                last_line.remove()
            trapezoid_with_point = current_node.label
            plot_trapezoid(trapezoid_with_point, color='blue', linestyle='-')
            ax.plot(point.x, point.y, marker='o', color='blue', markersize=8)
            break

    plt.show()
animation_map_point(lines, change_segment_representation(lines), Point(4,4))

## Analiza złożoności czasowej

czas budowy mapy trapezowej

In [ ]:
# Funkcja pomocnicza do mierzenia czasu
def measure_execution_time(lines):
    start = time.time()
    build_trapezoidal_map(lines)  # Wykonaj algorytm
    end = time.time()
    return end - start

# Główna analiza złożoności
sizes = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]  # Rozmiary wejściowe (liczba odcinków)
execution_times = []

for size in sizes:
      # Generuj odcinki
    # segmenty = change_segment_representation(odc) 
    # # Zmień reprezentację
    odc=generate_segments(size, (0,100), (0,100))

    time_taken = np.mean([measure_execution_time(odc) for _ in range(5)])
# Zmierz czas
    execution_times.append(time_taken)                # Zapisz czas

# Rysowanie wykresu
plt.figure(figsize=(10, 6))
plt.plot(sizes, execution_times, marker='o', label="Czas wykonania")
plt.title("Analiza złożoności algorytmu")
plt.xlabel("Liczba odcinków")
plt.ylabel("Czas wykonania (sekundy)")
plt.grid(True)
plt.legend()
plt.show()
print([round(execution_times[i],6) for i in range(len(execution_times))])


czas wyszukiwania punktu

In [42]:
def generate_parallel_segments(n, x_range, y_range):
    """
    Generuje n poziomych odcinków o różnych współrzędnych y.
    Przyspiesza proces testowania algorytmu.
    """
    segments = []
    set_y=set()
    for i in range(n):
        x1 = np.random.uniform(*x_range)
        x2=np.random.uniform(*x_range)
        y1 = np.random.uniform(*y_range)
        while y1 in set_y:
            y1+=1
        set_y.add(y1)
        segments.append(((x1, y1), (x2, y1)))
    return segments

In [ ]:
def measure_execution_time2(dtree):
    point=Point(random.uniform(0,10000),random.uniform(0,10000))

    start = time.time()
    find_point(dtree, point)
    # Wykonaj algorytm
    end = time.time()
    return end - start

# Główna analiza złożoności
sizes = [10,50,100,500,1000,5000,10000,50000,100000,500000]  # Rozmiary wejściowe (liczba odcinków)
execution_times = []
execution10000=[]

for size in sizes:
      # Generuj odcinki
    # segmenty = change_segment_representation(odc) 
    # # Zmień reprezentację
    odc=generate_parallel_segments(size, (0,10000), (0,10000))
    dtree=build_trapezoidal_map(odc) 

    time_taken = np.mean([measure_execution_time2(dtree) for _ in range(10000)])
    sum_time=sum([measure_execution_time2(dtree) for _ in range(10000)])
# Zmierz czas
    execution_times.append(time_taken) 
    execution10000.append(sum_time)               # Zapisz czas

# Rysowanie wykresu
plt.figure(figsize=(10, 6))
plt.plot(sizes, execution_times, marker='o', label="Czas wykonania")
plt.title("Analiza złożoności algorytmu")
plt.xlabel("Liczba odcinków")
plt.ylabel("Czas wykonania (sekundy)")
plt.grid(True)
plt.legend()
plt.show()
print([round(time,6) for time in execution10000])